#__🌋Disaster or Non Disaster Tweet🌪️__

Predicting Whether A Tweet Is About A Disaster

##1. Problem Statement
>Twitter has become an important communication channel in times of emergency. The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies). But, it’s not always clear whether a person’s words are actually announcing a disaster.

##2. Data
The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started

##3. Steps
 * Use Transfer learning in NLP
 * Use Universal Sentence Encoder
 * Train model with more than 80% accuracy
 * Predict on tweet from wild

In [2]:
!wget https://raw.githubusercontent.com/anmolmishra99/Subu/main/subu.py

from subu import unzip, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-02-24 16:26:35--  https://raw.githubusercontent.com/anmolmishra99/Subu/main/subu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4993 (4.9K) [text/plain]
Saving to: ‘subu.py’

subu.py             100%[===================>]   4.88K  --.-KB/s    in 0s      

2022-02-24 16:26:36 (42.1 MB/s) - ‘subu.py’ saved [4993/4993]



## Visualizing Data

In [3]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/disaster-nondisaster/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/disaster-nondisaster/test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
#shuffle taining dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [6]:
#Let's visualize some random training example
import random
random_index = random.randint(0, len(train_df) - 10) # create random index not higer than len(train_df)
for row in train_df_shuffled[['text', 'target']][random_index: random_index+10].itertuples(): # return tuple of text and target with index
  _, text, target = row
  print(f'target: {target}', '{real disaster}' if target > 0 else '{not real disaster}')
  print(f'Text:\n{text}\n\n')

target: 0 {not real disaster}
Text:
I also loved 'Bury my heart at wounded knee' too! #TheGame


target: 0 {not real disaster}
Text:
Smoke the Weed - Snoop Lion Ft. Collie Buddz


target: 1 {real disaster}
Text:
Prince Phillip said of the numbers of those murdered by the British at Amritsar...

ÛÏThatÛªs a bit exaggerated it must include the woundedÛ.


target: 0 {not real disaster}
Text:
#Tb #throwback ??

??~ You want a battle? Here's a War! ~ ?? https://t.co/B0ZJWgmaIW


target: 0 {not real disaster}
Text:
New Mad Max Screenshots Show Off a Lovely Dust Storm Combat Magnum Opus http://t.co/QHbzKErOTt #cogXbox #XboxOne #Xbox


target: 1 {real disaster}
Text:
Alarming Rise in Dead Marine Life Since the #Fukushima Nuclear Disaster: http://t.co/v6H97K688J http://t.co/tJw9bSeiPW


target: 1 {real disaster}
Text:
Good thing there was actually just a legit fire in the mall and nobody evacuated!!


target: 1 {real disaster}
Text:
11-Year-Old Boy Charged With Manslaughter of Toddler: Repo

In [7]:
from sklearn.model_selection import train_test_split
# user train_test_split to split training data to training and val sets
train_sentences, val_sentences, train_label, val_label = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size=0.1,
                                                                          random_state=42)

In [8]:
len(train_sentences), len(val_label)

(6851, 762)

## Tensorflow hub Pretrained Sentence Encoder

using USE(Universal Sentence Encoder)

In [9]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import layers 

sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                       input_shape = [], #Input of any length
                                       dtype=tf.string,
                                       trainable = False,
                                       name='USE') 

In [10]:
# Create model using the Seuential API
model = tf.keras.Sequential([
   sentence_encoder_layer,
   layers.Dense(64, activation='relu'),
   layers.Dense(1, activation='sigmoid'),
])

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [11]:
history = model.fit(train_sentences,
                        train_label,
                        epochs=10,
                        validation_data=(val_sentences, val_label),
                        callbacks=[create_tensorboard_callback('Tfboard', 'model_tfhub_use')])

Saving TensorBoard log files to: Tfboard/model_tfhub_use/20220224-162717
Epoch 1/10
215/215 [==============================] - 12s 21ms/step - loss: 0.5039 - accuracy: 0.7897 - val_loss: 0.4525 - val_accuracy: 0.7979
Epoch 2/10
215/215 [==============================] - 4s 19ms/step - loss: 0.4152 - accuracy: 0.8146 - val_loss: 0.4397 - val_accuracy: 0.8045
Epoch 3/10
215/215 [==============================] - 4s 19ms/step - loss: 0.4008 - accuracy: 0.8229 - val_loss: 0.4333 - val_accuracy: 0.8136
Epoch 4/10
215/215 [==============================] - 4s 17ms/step - loss: 0.3924 - accuracy: 0.8254 - val_loss: 0.4290 - val_accuracy: 0.8163
Epoch 5/10
215/215 [==============================] - 4s 19ms/step - loss: 0.3854 - accuracy: 0.8314 - val_loss: 0.4274 - val_accuracy: 0.8163
Epoch 6/10
215/215 [==============================] - 4s 18ms/step - loss: 0.3797 - accuracy: 0.8332 - val_loss: 0.4237 - val_accuracy: 0.8176
Epoch 7/10
215/215 [==============================] - 4s 19ms/step -

In [12]:
model_pred_probs = model.predict(val_sentences)
model_pred = tf.squeeze(tf.round(model_pred_probs))
model_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true, y_pred):
  '''
  claculate model accuracy, precision, recall and f1score od binary classification model
  '''
  # Calcualte model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # calculate model precision recall and f1score using "weighted"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {'accuracy': model_accuracy,
                   'presicion': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results


In [14]:
model_result = calculate_result(val_label, model_pred)
model_result

{'accuracy': 82.1522309711286,
 'f1': 0.8207007869609225,
 'presicion': 0.8221385434852317,
 'recall': 0.821522309711286}

##Predicting on Tweets from the wild

In [16]:
tweet = 'Someone save me there is earthquake in India'
pred_prob = model.predict([tweet])
pred = tf.squeeze(tf.round(pred_prob))
pred.numpy()

1.0

In [17]:
tweet = 'What\'s wrong with my life atm...'
pred_prob = model.predict([tweet])
pred = tf.squeeze(tf.round(pred_prob))
pred.numpy()

0.0